#### Explore some edge detection algorithms

In [1]:
# imports libraries
import os
import sys
import imp
import glob

import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
#import importlib
import math
import pandas as pd

%matplotlib inline 

# Grabs the preprocessing and automatic_sync files
sys.path.append(os.path.join(os.pardir,'pythonCode'))
import preprocessing as pp
import automatic_sync as autoS
from __future__ import division

In [2]:
import fracture_prediction as predict
imp.reload(predict)
imp.reload(autoS)

<module 'automatic_sync' from '../pythonCode/automatic_sync.py'>

In [3]:
# reading the data

rawDataPath = os.path.join(os.pardir,'rawAudio')
files = glob.glob(os.path.join(rawDataPath,'*.wav'))
names = []
        
for name in files:
    fileName = os.path.basename(name).split(".")[0]
    names.append(fileName)

# Determines which cameras will be selected (['Bents'],['Camera Location'],['Motion #'])

filt = (None,None,['18']) # Selects the bent 1 cameras during motion 18

# Applies filter to camera names and returns only selected subset names
audioFiles = pp.getKeys(names,filt);

# Reads the .wav files from the list generted by getKeys
(names,cDataset) = pp.readWAV(rawDataPath,audioFiles)

Opening GP_B1_NL_18 ...
Opening GP_B1_NU_18 ...
Opening GP_B1_SL_18 ...
Opening GP_B1_SU_18 ...
Opening GP_B2_SL_18 ...
Opening GP_B2_SU_18 ...
Opening GP_B3_NL_18 ...
Opening GP_B3_NU_18 ...
Opening GP_B3_SL_18 ...
Opening GP_B3_SU_18 ...


In [4]:
## syncing the dataset
final_offsets, sDataset = autoS.sync_dataset(cDataset,'GP_B1_NL_18',names,mask=[0,1])
#(dataset,indexName,names,max_iter = 2,mask=[0,1])
# print(autoS.find_offset(sDataset,names[0],names[1:]))

In [5]:
final_offsets

{'GP_B1_NL_18': 85198,
 'GP_B1_NU_18': 85082,
 'GP_B1_SL_18': 85041,
 'GP_B1_SU_18': 84693,
 'GP_B2_SL_18': 3018,
 'GP_B2_SU_18': 0,
 'GP_B3_NL_18': 80861,
 'GP_B3_NU_18': 81723,
 'GP_B3_SL_18': 81942,
 'GP_B3_SU_18': 82591}

In [6]:
# res = [features.edges2fractures(cDataset[key][:,0]) for key in cDataset.keys()]

In [ ]:
[k for k in cDataset.keys() if k != 'GP_B3_NL_18']

['GP_B3_SL_18',
 'GP_B2_SU_18',
 'GP_B3_NU_18',
 'GP_B2_SL_18',
 'GP_B1_NL_18',
 'GP_B1_SL_18',
 'GP_B3_SU_18',
 'GP_B1_NU_18',
 'GP_B1_SU_18']

In [ ]:
# extract fractures for each signal in the dataset (only the first channel)
fractureTimes = predict.applyFracturePredictor(sDataset, predict.edges2fractures, smoothing=5)

In [ ]:
# read Islam's estimates

import csv
print(os.pardir)
csvPath = os.path.join(os.pardir,'IslamEstimates')
fileName = glob.glob(os.path.join(csvPath,'GP_B1_NL_18.csv'))

fractInd = list(csv.reader(open(fileName[0], 'rt'), delimiter=','))
fractInd = np.array([float(i[0]) for i in fractInd]) # in counts
print(fractInd)

fractIM = 1/48000*(-final_offsets['GP_B1_NL_18'] + fractInd)
print(fractIM)

In [ ]:
# visualized the fracture estimates
index = 1
for name in names:
    plt.plot(fractureTimes[name], index*np.ones_like(fractureTimes[name]),'ko')
    index += 1
plt.plot([fractIM,fractIM],[0,len(names)+1],'r')
plt.yticks(np.arange(1,len(names)+1), names)
plt.xlim(30,40)
plt.show()

In [ ]:
# Calculate the dissimilarity measure
d = {}
for key in fractureTimes.keys():
    d[key] = (predict.dissimilarity(fractureTimes[key],fractIM,0.1,1.,'distance'))

In [ ]:
d